#### Libraries

In [1]:
import os
import re
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

from pymavlog import MavLog
from datetime import datetime

#### Class initialization

In [2]:
class PixhawkLogger(MavLog):
    
    def __init__(self, log_path: str):
        super().__init__(log_path)
        self.message = 'None'
        self.parse()

    @staticmethod
    def timestamp_ms(TimeUS):
        timestamp = TimeUS / 1_000
        return pd.to_datetime(timestamp, unit='ms')

    @staticmethod
    def timestamp_s(TimeUS):
        timestamp = TimeUS / 1_000_000
        return pd.to_datetime(timestamp, unit='s')

    def get_message(self, message: str):
        self.message = message

    def get_field(self, field: str):
        return self.get(self.message)[field]

    def get_all_messages(self, debug: bool=False):
        all_messages = self._types
 
        if debug:
            print("-- Available Log Messages --")
            for message in all_messages:
                print(f"- {message}")

        return all_messages

    def get_all_fields(self, debug: bool = False):
        if self.message not in self._parsed_data:
            raise ValueError(f"[ERROR] No Message: {self.message}")
        
        fields = list(self._parsed_data[self.message].columns)

        if debug:
            print(f"-- Available Fields in [{self.message}] --")
            print(fields)
        
        return fields

    def get_statustext(self, debug: bool=False):
        self.message = 'MSG'
        statustext = self.get_field('Message') 

        if debug:
            for text in statustext:
                print(text)

        return statustext

    def get_mode_data(self):
        self.message = 'MODE'
        time = self.get_field('TimeUS')

        df_mode = pd.DataFrame({
            'time'  : self.timestamp_ms(time),
            'mode'  : self.get_field('ModeNum')
        })

        return df_mode

    def get_msg_data(self):
        self.message = 'MSG'
        time = self.get_field('TimeUS')

        df_msg = pd.DataFrame({
            'time'  : self.timestamp_ms(time),
            'msg'  : self.get_field('Message')
        })

        return df_msg

    def get_gps_data(self):
        self.message = 'GPS'
        time = self.get_field('TimeUS')

        df_gps = pd.DataFrame({
            'time': self.timestamp_ms(time),
            'lat' : self.get_field('Lat'),
            'lon' : self.get_field('Lng'),
        })

        return df_gps

    def get_baro_data(self):
        self.message = 'BARO'
        time = self.get_field('TimeUS')

        df_baro = pd.DataFrame({
            'time': self.timestamp_ms(time),
            'alt' : self.get_field('Alt'),
        })

        return df_baro

    def get_att_data(self):
        self.message = 'ATT'
        time = self.get_field('TimeUS')

        df_att = pd.DataFrame({
            'time'  : self.timestamp_ms(time),
            'roll'  : self.get_field('Roll'),
            'pitch' : self.get_field('Pitch'),
            'yaw'   : self.get_field('Yaw')
        })

        return df_att

    def get_ctun_data(self):
        self.message = 'CTUN'
        time = self.get_field('TimeUS')

        df_ctun = pd.DataFrame({
            'time'  : self.timestamp_ms(time),
            'crt'  : self.get_field('CRt')
        })

        return df_ctun
    
    def get_imu_data(self):
        self.message = 'IMU'
        time = self.get_field('TimeUS')

        df_imu = pd.DataFrame({
            'time'  : self.timestamp_ms(time),
            'gyrX'  : self.get_field('GyrX'),
            'gyrY'  : self.get_field('GyrY'),
            'gyrZ'  : self.get_field('GyrZ'),
            'accX'  : self.get_field('AccX'),
            'accY'  : self.get_field('AccY'),
            'accZ'  : self.get_field('AccZ')
        })

        return df_imu

    def get_pwm_data(self):
        self.message = 'RCOU'
        time = self.get_field('TimeUS')

        df_pwm = pd.DataFrame({
            'time'  : self.timestamp_ms(time),
            'c1'  : self.get_field('C1'),
            'c2'  : self.get_field('C2'),
            'c3'  : self.get_field('C3'),
            'c4'  : self.get_field('C4')
        })

        return df_pwm

    def get_rpm_data(self):
        df_pwm = self.get_pwm_data().sort_values('time')
        # input model converter

    def get_merged_data(self):
        df_gps  = self.get_gps_data().sort_values('time')
        df_baro = self.get_baro_data().sort_values('time')
        df_att  = self.get_att_data().sort_values('time')
        df_imu  = self.get_imu_data().sort_values('time')
        df_ctun = self.get_ctun_data().sort_values('time')
        df_pwm  = self.get_pwm_data().sort_values('time')

        df_merged = pd.merge_asof(df_gps, df_baro, on='time', direction='nearest', tolerance=pd.Timedelta('500ms'))
        df_merged = pd.merge_asof(df_merged, df_att, on='time', direction='nearest', tolerance=pd.Timedelta('500ms'))
        df_merged = pd.merge_asof(df_merged, df_imu, on='time', direction='nearest', tolerance=pd.Timedelta('500ms'))
        df_merged = pd.merge_asof(df_merged, df_ctun, on='time', direction='nearest', tolerance=pd.Timedelta('500ms'))
        df_merged = pd.merge_asof(df_merged, df_pwm, on='time', direction='nearest', tolerance=pd.Timedelta('500ms'))

        return df_merged.sort_values('time')

    def get_merged_interpolate(self):
        df_gps  = self.get_gps_data()
        df_baro = self.get_barodata()
        df_att  = self.get_att_data()
        df_imu  = self.get_imu_data()
        df_ctun = self.get_ctun_data()
        df_pwm  = self.get_pwm_data()

        df_gps  = df_gps.set_index('time')
        df_baro = df_baroset_index('time').sort_index()
        df_att  = df_att.set_index('time').sort_index()
        df_imu  = df_imu.set_index('time').sort_index()
        df_ctun = df_ctun.set_index('time').sort_index()
        df_pwm = df_pwm.set_index('time').sort_index()

        df_baro_interp = df_baro.interpolate(method='time').reindex(df_gps.index, method='nearest')
        df_att_interp  = df_att.interpolate(method='time').reindex(df_gps.index, method='nearest')
        df_imu_interp  = df_imu.interpolate(method='time').reindex(df_gps.index, method='nearest')
        df_ctun_interp = df_ctun.interpolate(method='time').reindex(df_gps.index, method='nearest')
        df_pwm_interp  = df_pwm.interpolate(method='time').reindex(df_gps.index, method='nearest')

        df_merged = pd.concat([df_gps, df_baro_interp, df_att_interp, df_imu_interp, df_ctun_interp, df_pwm_interp], axis=1)
        df_merged = df_merged.reset_index().rename(columns={'index': 'time'})

        return df_merged

    def filter_circle(self):
        df_merged = self.get_merged_data()
        df_mode = self.get_mode_data()

        circle_mode = 7
        target_idx = df_mode[df_mode['mode'] == circle_mode].index

        if len(target_idx) == 0:
            raise ValueError("[WARNING] Mode circle tidak ada!")

        first_idx = target_idx[0]

        start_time = pd.to_datetime(df_mode.loc[first_idx - 1, 'time'])
        end_time   = pd.to_datetime(df_mode.loc[first_idx + 1, 'time'])

        return df_merged[(df_merged['time'] >= start_time) & (df_merged['time'] <= end_time)].copy().sort_values('time').reset_index(drop=True)
    
    def filter_lemni(self):
        df_merged = self.get_merged_data()
        df_msg = self.get_msg_data() 

        target_idx = df_msg[df_msg['msg'] == "Reached command #2"]

        if target_idx.empty:
            raise ValueError(f"[WARNING] Mission plan tidak ada!")

        start_time = pd.to_datetime(target_idx.iloc[0]['time'])

        return df_merged[df_merged['time'] >= start_time].sort_values('time').reset_index(drop=True)


    def plot_yaw(self):
        df = self.get_merged_data()

        plt.figure(figsize=(12, 6))
        plt.plot(df['time'], df['yaw'])
        plt.xlabel('Time')
        plt.ylabel('Yaw')
        plt.title('Yaw vs Time')
        plt.grid(True)
        plt.show()



#### Log circle

In [3]:
path = "log/circle_1.bin"

log = PixhawkLogger(path)

#### Dataframe all

In [4]:
df_all = log.get_merged_data()
df_all

,time,lat,lon,alt,roll,pitch,yaw,gyrX,gyrY,gyrZ,accX,accY,accZ,crt,c1,c2,c3,c4
0,1970-01-01 00:00:23.106147,-7.265868,112.791129,0.308003,0.57,-1.57,99.48,0.000809,-0.000424,0.015505,-0.161799,-0.128529,-9.829406,NaN,1000,1000,1000,1000
1,1970-01-01 00:00:23.226824,-7.265868,112.791129,0.144112,0.57,-1.57,99.50,-0.000386,-0.000353,-0.014057,-0.171069,-0.150347,-9.823412,NaN,1000,1000,1000,1000
2,1970-01-01 00:00:23.347885,-7.265868,112.791129,0.152589,0.57,-1.57,99.50,0.000674,-0.001611,0.014841,-0.178722,-0.149731,-9.819493,NaN,1000,1000,1000,1000
3,1970-01-01 00:00:23.408929,-7.265867,112.791129,0.152589,0.57,-1.58,99.52,0.000795,-0.000023,0.003978,-0.174291,-0.159833,-9.815162,NaN,1000,1000,1000,1000
4,1970-01-01 00:00:23.508856,-7.265867,112.791129,0.121506,0.57,-1.57,99.47,-0.000088,-0.000154,0.003808,-0.171710,-0.126487,-9.830187,NaN,1000,1000,1000,1000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
827,1970-01-01 00:02:02.628853,-7.265868,112.791100,-0.082484,1.03,-0.36,93.22,0.000377,0.000317,0.001816,-0.022230,-0.240980,-9.815374,NaN,1000,1000,1000,1000
828,1970-01-01 00:02:02.728946,-7.265868,112.791100,0.070104,1.03,-0.36,93.22,0.000553,-0.000720,0.001275,-0.030813,-0.256754,-9.817620,NaN,1000,1000,1000,1000
829,1970-01-01 00:02:02.828849,-7.265868,112.791100,-0.054227,1.03,-0.37,93.22,0.000048,-0.000832,0.001458,-0.028595,-0.248790,-9.824906,NaN,1000,1000,1000,1000
830,1970-01-01 00:02:02.928849,-7.265868,112.791100,0.064453,1.03,-0.36,93.23,-0.000041,-0.000845,0.001655,-0.030963,-0.244207,-9.833687,NaN,1000,1000,1000,1000


#### Dataframe filter

In [5]:
log.get_mode_data()

,time,mode
0,1970-01-01 00:00:23.478927000,2
1,1970-01-01 00:00:47.571860000,2
2,1970-01-01 00:01:06.806047999,7
3,1970-01-01 00:01:30.013637000,2


In [6]:
df_filter = log.filter_circle()
df_filter

,time,lat,lon,alt,roll,pitch,yaw,gyrX,gyrY,gyrZ,accX,accY,accZ,crt,c1,c2,c3,c4
0,1970-01-01 00:00:47.648827,-7.265869,112.791132,-0.110742,0.63,-1.63,76.10,0.001268,0.001648,0.005566,-0.187732,-0.157773,-9.826408,NaN,1000,1000,1000,1000
1,1970-01-01 00:00:47.729154,-7.265869,112.791132,-0.348102,0.63,-1.63,76.11,0.000754,0.001606,-0.000760,-0.172226,-0.153335,-9.825445,NaN,1000,1000,1000,1000
2,1970-01-01 00:00:47.829073,-7.265869,112.791132,-0.090962,0.63,-1.62,76.13,0.000978,-0.001453,0.005315,-0.194289,-0.152275,-9.807342,NaN,1000,1000,1000,1000
3,1970-01-01 00:00:47.928947,-7.265869,112.791132,-0.170082,0.63,-1.62,76.15,0.001738,-0.002411,-0.000475,-0.182186,-0.154359,-9.823212,NaN,1000,1000,1000,1000
4,1970-01-01 00:00:48.029032,-7.265869,112.791132,-0.037273,0.62,-1.62,76.17,-0.000990,-0.001859,0.005081,-0.184739,-0.133487,-9.822755,NaN,1000,1000,1000,1000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
387,1970-01-01 00:01:29.529142,-7.265857,112.791116,2.291120,-1.28,-1.56,87.50,-0.009860,0.034678,0.300431,-0.008383,0.153342,-9.860460,NaN,1492,1484,1541,1579
388,1970-01-01 00:01:29.629163,-7.265856,112.791116,2.488920,-1.19,-1.53,89.20,0.036961,-0.105073,0.329439,0.063044,0.139118,-9.706987,NaN,1519,1453,1600,1515
389,1970-01-01 00:01:29.729260,-7.265855,112.791116,2.262863,-1.09,-1.62,90.94,0.022569,0.055622,0.292275,-0.070716,0.136213,-9.905859,NaN,1508,1463,1560,1555
390,1970-01-01 00:01:29.829140,-7.265855,112.791116,2.180917,-1.04,-1.62,92.68,0.039615,0.036749,0.305708,-0.067327,0.163721,-9.909031,NaN,1515,1454,1597,1513


#### Log lemni

In [7]:
path = "log/lemni_1.bin"

logs = PixhawkLogger(path)

#### Dataframe all

In [8]:
df_all = logs.get_merged_data()
df_all

,time,lat,lon,alt,roll,pitch,yaw,gyrX,gyrY,gyrZ,accX,accY,accZ,crt,c1,c2,c3,c4
0,1970-01-01 00:02:37.925412000,-7.265877,112.791078,0.038434,0.65,-0.91,87.39,-0.000482,-0.001765,-0.001155,-0.063218,-0.165814,-9.798840,NaN,1000,1000,1000,1000
1,1970-01-01 00:02:38.025934000,-7.265877,112.791078,0.038434,0.65,-0.91,87.39,-0.001011,-0.000583,-0.002782,-0.062686,-0.158331,-9.783065,NaN,1000,1000,1000,1000
2,1970-01-01 00:02:38.127871000,-7.265877,112.791078,-0.000878,0.65,-0.91,87.39,0.001036,-0.001729,-0.000435,-0.065899,-0.160682,-9.802204,NaN,1000,1000,1000,1000
3,1970-01-01 00:02:38.209129000,-7.265876,112.791078,0.010354,0.66,-0.92,87.39,0.000536,-0.000369,-0.002279,-0.067912,-0.163927,-9.800107,NaN,1000,1000,1000,1000
4,1970-01-01 00:02:38.310598000,-7.265876,112.791078,0.021586,0.66,-0.92,87.39,0.000150,-0.000689,-0.001945,-0.066356,-0.144809,-9.789960,NaN,1000,1000,1000,1000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1074,1970-01-01 00:04:25.572613000,-7.265855,112.791146,0.046858,0.24,-0.42,82.85,0.004651,-0.004296,-0.002677,0.021408,-0.077122,-9.806375,NaN,1000,1000,1000,1000
1075,1970-01-01 00:04:25.652385000,-7.265855,112.791146,0.083362,0.25,-0.42,82.85,0.000713,-0.000447,-0.002369,0.019733,-0.092676,-9.806516,NaN,1000,1000,1000,1000
1076,1970-01-01 00:04:25.752381000,-7.265855,112.791146,-0.006494,0.24,-0.42,82.85,-0.000442,-0.001418,-0.002162,0.025483,-0.074516,-9.807624,NaN,1000,1000,1000,1000
1077,1970-01-01 00:04:25.832697999,-7.265855,112.791146,-0.113197,0.24,-0.41,82.85,-0.000030,-0.000729,0.000201,0.018730,-0.074743,-9.798866,NaN,1000,1000,1000,1000


#### Dataframe filter

In [9]:
df_wp = logs.get_msg_data()
df_wp

,time,msg
0,1970-01-01 00:02:38.250141,ArduCopter V4.5.7 (2a3dc4b7)
1,1970-01-01 00:02:38.250171,ChibiOS: 6a85082c
2,1970-01-01 00:02:38.250308,Pixhawk1 003C001A 3532510C 34303430
3,1970-01-01 00:02:38.250352,Param space used: 850/4096
4,1970-01-01 00:02:38.250381,RC Protocol: CRSF
...,...,...
110,1970-01-01 00:04:00.763181,Reached command #50
111,1970-01-01 00:04:00.763320,Mission: 51 WP
112,1970-01-01 00:04:02.571458,Reached command #51
113,1970-01-01 00:04:02.571532,Mission: 52 Land


In [10]:
df_wp.loc[14]

time    1970-01-01 00:02:58.099654
msg             Reached command #2
Name: 14, dtype: object

In [12]:
df_filter = logs.filter_lemni()
df_filter

,time,lat,lon,alt,roll,pitch,yaw,gyrX,gyrY,gyrZ,accX,accY,accZ,crt,c1,c2,c3,c4
0,1970-01-01 00:02:58.130123000,-7.265859,112.791146,1.891703,-0.59,0.38,89.76,0.009308,-0.154733,-0.002869,0.007455,0.170835,-9.801802,NaN,1387,1476,1484,1581
1,1970-01-01 00:02:58.230423000,-7.265859,112.791146,1.838352,-0.74,0.04,89.66,-0.042781,0.001309,-0.041324,-0.079032,-0.016847,-9.810568,NaN,1477,1384,1563,1498
2,1970-01-01 00:02:58.330419000,-7.265858,112.791146,1.953479,-0.72,-0.37,89.54,-0.005847,0.121650,-0.032467,-0.214931,0.011112,-9.925270,NaN,1403,1462,1510,1549
3,1970-01-01 00:02:58.430437000,-7.265858,112.791146,2.150038,-0.65,-0.50,89.38,0.046658,-0.136117,-0.005746,0.011722,0.094356,-9.728750,NaN,1472,1394,1519,1538
4,1970-01-01 00:02:58.550486000,-7.265858,112.791146,2.158462,-0.36,-0.52,89.18,0.026364,0.037649,-0.042660,-0.051549,-0.012909,-9.754099,NaN,1425,1447,1521,1535
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
872,1970-01-01 00:04:25.572613000,-7.265855,112.791146,0.046858,0.24,-0.42,82.85,0.004651,-0.004296,-0.002677,0.021408,-0.077122,-9.806375,NaN,1000,1000,1000,1000
873,1970-01-01 00:04:25.652385000,-7.265855,112.791146,0.083362,0.25,-0.42,82.85,0.000713,-0.000447,-0.002369,0.019733,-0.092676,-9.806516,NaN,1000,1000,1000,1000
874,1970-01-01 00:04:25.752381000,-7.265855,112.791146,-0.006494,0.24,-0.42,82.85,-0.000442,-0.001418,-0.002162,0.025483,-0.074516,-9.807624,NaN,1000,1000,1000,1000
875,1970-01-01 00:04:25.832697999,-7.265855,112.791146,-0.113197,0.24,-0.41,82.85,-0.000030,-0.000729,0.000201,0.018730,-0.074743,-9.798866,NaN,1000,1000,1000,1000
